In [1]:
import pandas as pd
import numpy as np
import json
import urllib
import time

In [2]:
master_artists = pd.read_csv("master_artists_list_latest.csv")

In [3]:
master_artists.columns

Index(['artist_name', 'age', 'years_active', 'genre_pop', 'genre_rock',
       'genre_hip_hop', 'genre_world_music', 'genre_r_and_b',
       'genre_spiritual', 'genre_electronic', 'genre_jazz', 'genre_country',
       'genre_blues', 'genre_folk', 'facebook', 'instagram', 'twitter',
       'youtube', 'musicbrainz'],
      dtype='object')

In [12]:
df = pd.DataFrame(columns = ['artist_name','title','release_date','primary_type'])

In [14]:
for index, row in master_artists.iterrows():
    time.sleep(5)
    offset = 0
    while True:
        url = "http://musicbrainz.org/ws/2/release/?query=arid:" + row['musicbrainz']+ "&fmt=json&inc=release-groups&limit=100&offset=" + str(offset)
        contents = urllib.request.urlopen(url).read()
        channels = json.loads(contents)
        offset += 100
        if len(channels['releases']) == 0:
            break
        for releases in channels['releases']:
            features = []
            features.append(row['artist_name'])
            if 'title' in releases:
                features.append(releases['title'])
            else:
                features.append(np.NAN)

            if 'date' in releases:
                features.append(releases['date'])
            else:
                features.append(np.NAN)

            if 'release-group' in releases:
                if 'primary-type' in releases['release-group']:
                    features.append(releases['release-group']['primary-type'])
                else:
                    features.append(np.NAN)
            else:
                features.append(np.NAN)
            df = df.append(pd.Series(features,index = df.columns),ignore_index=True)

In [15]:
df.count()

artist_name     20752
title           20752
release_date    18157
primary_type    20583
dtype: int64

In [16]:
df['release_date'].isnull().values.sum()

2595

In [17]:
df = df.dropna(axis=0, subset=['release_date'])

In [18]:
df.count()

artist_name     18157
title           18157
release_date    18157
primary_type    18084
dtype: int64

In [19]:
df.dtypes

artist_name     object
title           object
release_date    object
primary_type    object
dtype: object

In [20]:
df['release_date'].str.len().unique()

array([ 4, 10,  7,  0], dtype=int64)

In [21]:
df = df[df['release_date'].str.len() != 0]

In [22]:
df.count()

artist_name     17611
title           17611
release_date    17611
primary_type    17544
dtype: int64

In [23]:
df[df['release_date'].str.len() == 4].release_date.describe()

count     5752
unique      60
top       2001
freq       258
Name: release_date, dtype: object

In [25]:
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d %H:%M:%S')

In [26]:
df.groupby(['artist_name','title']).count()

release_date  \
artist_name title                                                              
21 Savage   1Night                                                         1   
            Bartier Cardi                                                  2   
            Enzo                                                           1   
            Focus                                                          2   
            Free Guwop EP                                                  1   
            Gucci on My                                                    1   
            How To Ball                                                    1   
            Issa Album                                                     1   
            Krippy Kush (remix)                                            1   
            One Foot                                                       1   
            Red Opps                                                       1   
            Rover 2.0                                                      1   
            Savage Mode                                                    1   
            Slaughter King                                                 2   
            Sneakin’                                                       1   
            The Slaughter Tape                                             1   
            Without Warning                                                2   
            i am > i was                                                   3   
AC/DC       '74 Jailbreak                                                 10   
            1976-12-18: A Giant Doze of Rock 'n' Roll: Fest...             1   
            1979-11-12: Jaap Edenhal, Amsterdam, The Nether...             1   
            1995-01-15: Rock in Rio: Cidade do Rock, Rio de...             1   
            1996 Ballbreaker Australian Tour EP                            1   
            2003-03-07: The Howard Stern Show: New York, NY...             2   
            2003-06-22: Sympathy For The Devil: Live am 22....             2   
            2009-11-27: Black Ice Tour: Morumbi Stadium, Sã...             2   
            3 Live Tracks                                                  1   
            3 Record Set                                                   2   
            5 Titres inédits en concert                                    1   
            A Vulgar Display of Ultra Rare Tracks                          2   
...                                                                      ...   
Zayn        Back to Sleep (remix)                                          1   
            BeFoUr                                                         1   
            Cruel                                                          1   
            Cruel (Jay Pryor remix)                                        1   
            Cruel (MXXWLL remix)                                           1   
            Cruel (Ta-ku remix)                                            1   
            Dusk Till Dawn                                                 1   
            Dusk Till Dawn (The Remixes)                                   1   
            Dusk Till Dawn (radio edit)                                    1   
            Entertainer                                                    1   
            Fingers                                                        1   
            Freedun                                                        1   
            I Don't Wanna Live Forever (Fifty Shades Darker)               1   
            I Don’t Wanna Live Forever (Fifty Shades Darker)               1   
            Icarus Falls                                                   5   
            LIKE I WOULD                                                   1   
            LIKE I WOULD (The Remixes)                                     1   
            LIKE I WOULD (The White Panda remix)                

In [27]:
df.dtypes

artist_name             object
title                   object
release_date    datetime64[ns]
primary_type            object
dtype: object

In [28]:
df.sort_values(by=['artist_name','title','release_date'],inplace=True)

In [29]:
df.head()

,artist_name,title,release_date,primary_type
9378,21 Savage,1Night,2018-04-27,Single
9380,21 Savage,Bartier Cardi,2017-12-22,Single
9383,21 Savage,Bartier Cardi,2017-12-22,Single
9370,21 Savage,Enzo,2016-04-24,Single
9381,21 Savage,Focus,2019-06-13,Single


In [30]:
df = df.groupby(['artist_name','title']).first()

In [31]:
df.reset_index(inplace=True)

In [32]:
df.head()

,artist_name,title,release_date,primary_type
0,21 Savage,1Night,2018-04-27,Single
1,21 Savage,Bartier Cardi,2017-12-22,Single
2,21 Savage,Enzo,2016-04-24,Single
3,21 Savage,Focus,2019-06-13,Single
4,21 Savage,Free Guwop EP,2015-07-02,Album


In [33]:
df.count()

artist_name     8511
title           8511
release_date    8511
primary_type    8459
dtype: int64

In [34]:
df.to_csv('musicbrainz_data.csv')